In [1]:
# 1. 표준 라이브러리
import sys
from pathlib import Path
import platform
import time
import json
import os
import joblib
from datetime import datetime
from typing import Optional
from dataclasses import dataclass


# 2. 서드파티 라이브러리 

# 2-1. 시각화
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

# 2-2. 
import shap
from imblearn.pipeline import Pipeline as ImbPipeline
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, make_scorer, recall_score, 
    precision_score, f1_score, fbeta_score, average_precision_score, balanced_accuracy_score, precision_recall_fscore_support
)
from scipy.stats import uniform, randint
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.base import BaseEstimator

sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR, MODEL_DIR

/Users/eric/yeeun/anaconda3/envs/smartfarm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [3]:
pandas_kwargs = {
    'parse_dates': ['검정일자'],
    'date_format': '%Y-%m-%d'
}

milk: pd.DataFrame = pd.read_csv(DATA_DIR /'interim' / 'milk.csv', **pandas_kwargs)
milk.head()

,농장아이디,개체번호,검정일자,누적착유일(연계),무지고형분,우유내요소태질소,305일유량,305일무지고형분,전산차비유지속성,전산차건유전유량,...,농후사료급여량(연계),공태일수,비유단계,계절,농장구분,정액코드분류,가격,가격미달,분만간격,분만월령
0,20249,20120709020022,2020-01-11,154,0,1,9714.0,805.0,50.5,16.1,...,0.0,NaN,1,4,1,NaN,1056.34,0,NaN,85
1,20249,20120504020095,2020-01-11,52,0,1,NaN,NaN,68.8,22.6,...,0.0,NaN,1,4,1,NaN,1056.34,0,NaN,90
2,20249,20111008020210,2020-01-11,115,0,1,11008.0,971.0,54.1,24.5,...,0.0,NaN,2,4,1,NaN,1060.34,0,NaN,95
3,20249,20121014020049,2020-01-11,290,1,1,11318.0,1011.0,58.6,37.8,...,0.0,224.0,3,4,1,7HO,1062.31,0,NaN,77
4,20249,20130812020216,2020-01-11,100,0,1,11324.0,957.0,68.1,20.5,...,0.0,NaN,2,4,1,NaN,1056.34,0,NaN,75


In [4]:
df = milk

In [5]:
features = ['전산차건유전유량', '산차', '농후사료급여량(연계)', '공태일수', '계절', '농장구분', '정액코드분류','분만간격']

target = '가격미달'

## 범주형 처리

In [6]:
categorical_features = ['정액코드분류']

for categorical_feature in categorical_features:
    df[categorical_feature] = df[categorical_feature].astype('category')

## Train Test Split

In [7]:
train = df[df['검정일자'].dt.year == 2020]
test = df[df['검정일자'].dt.year == 2021]

X_train, X_test = train[features], test[features]
y_train, y_test = train[target], test[target]

In [ ]:

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pkg_resources")

# ============================================
# 1. 클래스 비율 확인
# ============================================
print("="*60)
print("클래스 분포 분석")
print("="*60)
print(y_train.value_counts().sort_index())
print(f"\n비율:")
print(y_train.value_counts(normalize=True).sort_index())

# ✅ 수정: 0이 일반가(다수), 1이 가격미달(소수)
majority_count = (y_train == 0).sum()   # 일반가 (다수) - 약 90%
minority_count = (y_train == 1).sum()   # 가격미달 (소수) - 약 10%

print(f"\n클래스 구성:")
print(f"   0 (일반가, 다수): {majority_count}개 ({majority_count/len(y_train)*100:.1f}%)")
print(f"   1 (가격미달, 소수): {minority_count}개 ({minority_count/len(y_train)*100:.1f}%)")

# ✅ scale_pos_weight: 양성 클래스(1=가격미달=소수)에 대한 가중치
# 공식: (다수 클래스 개수) / (소수 클래스 개수)
scale_pos_weight = majority_count / minority_count

print(f"\n계산된 scale_pos_weight: {scale_pos_weight:.2f}")
print(f"   → LightGBM은 자동으로 클래스 1(가격미달)에 이 가중치 적용")

# 결측치 확인
print("\n결측치 현황:")
missing_info = pd.DataFrame({
    'Feature': X_train.columns,
    '결측_개수': X_train.isnull().sum().values,
    '결측_비율(%)': (X_train.isnull().mean() * 100).values
}).sort_values('결측_비율(%)', ascending=False)
print(missing_info.head(10))
print(f"\n전체 결측 비율: {X_train.isnull().mean().mean()*100:.1f}%")
print(f"Feature 개수: {X_train.shape[1]}개")

# ============================================
# 2. Pipeline 구성
# ============================================

pipeline = ImbPipeline([
    ('model', LGBMClassifier(
        objective='binary',
        device='cpu',
        random_state=42,
        verbosity=-1,
        n_jobs=-1,
        scale_pos_weight=scale_pos_weight,  # ✅ 자동으로 pos_label=1에 적용
        force_col_wise=True,
        metric='None',  # 🔥 추가: 기본 메트릭 끄기
        # 범주형 관련 파라미터
        cat_smooth=10,
        cat_l2=10,
        max_cat_threshold=32,
        max_cat_to_onehot=4
    ))
])

print("\n✅ Pipeline 구성:")
print("   1. LGBMClassifier (GPU)")
print(f"      - scale_pos_weight={scale_pos_weight:.2f} (클래스 1=가격미달 가중치)")

# ============================================
# 3. 파라미터 분포
# ============================================
print(f"\n소수 클래스(1=가격미달) 샘플 수: {minority_count}개")

param_distributions = {
    # 🔥 LightGBM 파라미터 - 소수 클래스 민감도 향상
    'model__n_estimators': randint(300, 1000),
    'model__learning_rate': uniform(0.005, 0.045),
    'model__max_depth': randint(3, 8),
    'model__num_leaves': randint(15, 80),
    'model__min_child_samples': randint(5, 50),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4),
    'model__min_child_weight': [1e-5, 1e-4, 1e-3, 1e-2],
    'model__reg_alpha': uniform(0, 0.5),
    'model__reg_lambda': uniform(0, 0.5),
}


# ============================================
# 4. Scoring 설정
# ============================================

scoring = {
    'recall': make_scorer(recall_score),
    'f2': make_scorer(fbeta_score, beta=2),              
    'pr_auc': make_scorer(average_precision_score, response_method='predict_proba'),
    'precision': make_scorer(precision_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0, average='macro'),
    'roc_auc': 'roc_auc'                                           
}

main_score = 'pr_auc'

print("\n✅ Scoring 전략:")
print(f"   메인 목표: {main_score.upper()}")
print(f"   타깃: 클래스 1 (가격미달) 검출 최적화")
print(f"   pos_label: 1 (기본값, 생략 가능)")

# ============================================
# 5. Stratified K-Fold
# ============================================
n_folds = 5
n_iter = 150
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)


print(f"\n교차검증 Folds: {n_folds}")
print(f"랜덤 샘플링 조합 수: {n_iter}개")
print(f"총 Fits: {n_iter} × {n_folds} = {n_iter * n_folds}")

# ============================================
# 6. RandomizedSearchCV 실행
# ============================================

print("\n" + "="*60)
print("🚀 RandomizedSearchCV 시작!")
print("="*60)

lgbm_random = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring,
    refit=main_score,
    verbose=3,
    random_state=42,
    n_jobs=-1,
    error_score=np.nan 
)

start_actual = time.time()
lgbm_random.fit(X_train, y_train)
actual_time = time.time() - start_actual

# ============================================
# 7. 결과 출력
# ============================================
print("\n" + "="*60)
print("✅ 완료!")
print("="*60)
print(f"⏱️  소요 시간: {actual_time/60:.1f}분 ({actual_time:.0f}초)")

print(f"\n🏆 최고 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")

print("\n📋 최적 파라미터:")
for key, value in sorted(lgbm_random.best_params_.items()):
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")
    else:
        print(f"   {key}: {value}")


클래스 분포 분석
가격미달
0    16343
1     1794
Name: count, dtype: int64

비율:
가격미달
0    0.901086
1    0.098914
Name: proportion, dtype: float64

클래스 구성:
   0 (일반가, 다수): 16343개 (90.1%)
   1 (가격미달, 소수): 1794개 (9.9%)

계산된 scale_pos_weight: 9.11
   → LightGBM은 자동으로 클래스 1(가격미달)에 이 가중치 적용

결측치 현황:
       Feature  결측_개수   결측_비율(%)
7         분만간격  14401  79.401224
6       정액코드분류   8907  49.109555
3         공태일수   8481  46.760765
0     전산차건유전유량   7340  40.469758
1           산차      0   0.000000
2  농후사료급여량(연계)      0   0.000000
4           계절      0   0.000000
5         농장구분      0   0.000000

전체 결측 비율: 27.0%
Feature 개수: 8개

✅ Pipeline 구성:
   1. LGBMClassifier (GPU)
      - scale_pos_weight=9.11 (클래스 1=가격미달 가중치)

소수 클래스(1=가격미달) 샘플 수: 1794개

✅ Scoring 전략:
   메인 목표: PR_AUC
   타깃: 클래스 1 (가격미달) 검출 최적화
   pos_label: 1 (기본값, 생략 가능)

교차검증 Folds: 5
랜덤 샘플링 조합 수: 150개
총 Fits: 150 × 5 = 750

🚀 RandomizedSearchCV 시작!
Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV 5/5] END model__colsample_bytree=0

In [ ]:

# ============================================
# 8. 전체 결과 비교
# ============================================
results_df = pd.DataFrame(lgbm_random.cv_results_)

scoring_cols = [f'mean_test_{score}' for score in scoring.keys()]

comparison_cols = [f'rank_test_{main_score}'] + scoring_cols + ['params']

print("\n" + "="*60)
print(f"📊 상위 10개 파라미터 조합 ({main_score.upper()} 기준)")
print("="*60)
top_10 = results_df[comparison_cols].sort_values(f'rank_test_{main_score}').head(10)

for idx, row in top_10.iterrows():
    rank = int(row[f'rank_test_{main_score}'])
    pr_auc = row[f'mean_test_pr_auc']
    recall = row['mean_test_recall']
    f2 = row['mean_test_f2']
    precision = row['mean_test_precision']
    f1 = row['mean_test_f1']
    
    print(f"\n#{rank} - {main_score.upper()}: {row[f'mean_test_{main_score}']:.4f} | Recall: {recall:.4f}")
    print(f"   F2: {f2:.3f} | Precision: {precision:.3f} | F1: {f1:.3f}")
    
    params = row['params']
    print(f"   Model: lr={params.get('model__learning_rate', 0):.4f}, "
          f"depth={params.get('model__max_depth', 0)}, "
          f"leaves={params.get('model__num_leaves', 0)}")

# ============================================
# 9. 테스트 세트 평가
# ============================================
print("\n" + "="*60)
print("🧪 테스트 세트 평가")
print("="*60)

best_model = lgbm_random.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# ✅ 클래스 1(가격미달)의 확률 추출
y_pred_proba_minority = y_pred_proba[:, 1]  # ✅ 인덱스 1 = 클래스 1 = 가격미달

print(classification_report(
    y_test, y_pred, 
    target_names=['0 (일반가, 다수)', '1 (가격미달, 소수)']
))

print("\n혼동 행렬:")
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index=['실제_0 (일반가)', '실제_1 (가격미달)'],
                     columns=['예측_0 (일반가)', '예측_1 (가격미달)'])
print(cm_df)

# ✅ 혼동 행렬 해석 수정
# confusion_matrix 구조:
#              예측_0  예측_1
# 실제_0 (일반가)  TN      FP
# 실제_1 (가격미달)   FN      TP

tn, fp, fn, tp = cm.ravel()

print("\n💼 비즈니스 해석 (소수 클래스=1=가격미달 기준):")
print(f"   ✅ 정확한 분류: {tn + tp}개 ({(tn+tp)/len(y_test)*100:.1f}%)")
print(f"   ⚠️  오탐 (FP): {fp}개 → 일반가을 가격미달로 오판 (재검사 필요)")
print(f"   🚨 미탐 (FN): {fn}개 → 가격미달를 일반가으로 오판 (치명적!)")

# ✅ Recall 계산: TP / (TP + FN)
if tp + fn > 0:
    recall_pct = tp/(tp+fn)*100
    print(f"\n   📊 소수 클래스(1=가격미달) Recall: {recall_pct:.1f}%")
    print(f"      → 가격미달 {tp+fn}개 중 {tp}개 검출 ({fn}개 놓침)")
else:
    print(f"\n   📊 소수 클래스(1=가격미달) Recall: N/A")

# ✅ Precision 계산: TP / (TP + FP)
if tp + fp > 0:
    precision_pct = tp/(tp+fp)*100
    print(f"   📊 소수 클래스(1=가격미달) Precision: {precision_pct:.1f}%")
    print(f"      → 가격미달 판정 {tp+fp}개 중 {tp}개 정답")
else:
    print(f"   📊 소수 클래스(1=가격미달) Precision: N/A")

test_roc_auc = roc_auc_score(y_test, y_pred_proba_minority)
print(f"   📊 ROC-AUC: {test_roc_auc:.4f}")

# ============================================
# 10. 🔥 임계값 조정으로 추가 개선
# ============================================
from sklearn.metrics import precision_recall_curve, average_precision_score

# ✅ pos_label=1 (가격미달) 기준으로 PR 곡선 계산
precisions, recalls, thresholds = precision_recall_curve(
    y_test, y_pred_proba_minority
)

# Recall 80% 보장
target_recall = 0.80
idx = np.where(recalls >= target_recall)[0]

if len(idx) > 0:
    best_idx = idx[np.argmax(precisions[idx])]
    optimal_threshold = thresholds[best_idx]
    
    print("\n" + "="*60)
    print(f"🎯 Recall {target_recall*100}% 보장 시 최적 임계값")
    print("="*60)
    print(f"최적 임계값: {optimal_threshold:.3f} (기본값 0.5)")
    print(f"Precision: {precisions[best_idx]:.3f}")
    print(f"Recall: {recalls[best_idx]:.3f}")
    
    # ✅ 임계값 조정: 확률 >= threshold이면 클래스 1(가격미달)로 예측
    y_pred_adjusted = (y_pred_proba_minority >= optimal_threshold).astype(int)
    
    cm_adj = confusion_matrix(y_test, y_pred_adjusted)
    tn2, fp2, fn2, tp2 = cm_adj.ravel()
    
    print(f"\n조정 효과:")
    print(f"   미탐(FN) 감소: {fn} → {fn2} (개선: {fn-fn2}개)")
    print(f"   오탐(FP) 증가: {fp} → {fp2} (증가: {fp2-fp}개)")
    
    if tp + fn > 0 and tp2 + fn2 > 0:
        print(f"   Recall 개선: {tp/(tp+fn)*100:.1f}% → {tp2/(tp2+fn2)*100:.1f}%")
    
    if fn > fn2:
        print(f"   💡 가격미달를 일반가으로 오판하는 치명적 오류 {fn-fn2}개 감소!")
else:
    print(f"\n⚠️  Recall {target_recall*100}% 달성 불가 (최대 Recall: {recalls.max():.2%})")

# ============================================
# 11. 변수 중요도 확인
# ============================================
print("\n" + "="*60)
print("📊 변수 중요도 Top 10")
print("="*60)

final_model = best_model.named_steps['model']
feature_names = list(X_train.columns)

n_features_used = len(final_model.feature_importances_)
feature_names = feature_names[:n_features_used]

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(importance_df.head(10).to_string(index=False))

# ============================================
# 12. 교차검증 vs 테스트 성능 비교
# ============================================
print("\n" + "="*60)
print("📈 성능 비교: 교차검증 vs 테스트")
print("="*60)

# ✅ pos_label=1 기본값 사용
test_metrics = {
    'PR-AUC': average_precision_score(y_test, y_pred_proba_minority),
    'Recall': recall_score(y_test, y_pred),
    'F2': fbeta_score(y_test, y_pred, beta=2),
    'Precision': precision_score(y_test, y_pred, zero_division=0),
    'F1': f1_score(y_test, y_pred, zero_division=0),
    'ROC-AUC': test_roc_auc
}

print(f"{'지표':<15} {'교차검증 (CV)':<20} {'테스트':<15} {'차이':<10}")
print("-" * 60)

for metric in ['PR-AUC', 'Recall', 'F2', 'Precision', 'F1', 'ROC-AUC']:
    cv_key = metric.lower().replace('-', '_')
    cv_score = results_df.loc[results_df[f'rank_test_{main_score}'] == 1, f'mean_test_{cv_key}'].values[0]
    test_score = test_metrics[metric]
    diff = test_score - cv_score
    
    diff_str = f"{diff:+.4f}"
    if abs(diff) > 0.05:
        diff_str += " ⚠️"
    
    print(f"{metric:<15} {cv_score:<20.4f} {test_score:<15.4f} {diff_str}")

# ============================================
# 13. 최종 요약
# ============================================
print("\n" + "="*60)
print(f"📝 최종 요약 ({main_score.upper()} 최적화)")
print("="*60)
print(f"✅ 최적 모델 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")
print(f"✅ 테스트 Recall: {test_metrics['Recall']*100:.1f}% (가격미달 검출률)")
print(f"✅ 테스트 Precision: {test_metrics['Precision']*100:.1f}% (가격미달 판정 정확도)")
print(f"✅ 테스트 F1: {test_metrics['F1']:.4f}")
print(f"✅ 치명적 오류(미탐 FN): {fn}개 / {tp+fn}개")
print(f"⏱️  총 소요 시간: {actual_time/60:.1f}분")

print("\n💡 핵심 포인트:")
print(f"   - 가격미달(1) {tp+fn}개 중 {tp}개 검출 ({tp/(tp+fn)*100:.1f}%)")
print(f"   - 놓친 가격미달: {fn}개")
print(f"   - 오탐 (일반가→가격미달): {fp}개 (재검사 필요)")

# ============================================
# 14. 모델 + 파라미터 저장
# ============================================
print("\n" + "="*60)
print("💾 모델 및 파라미터 저장")
print("="*60)

save_model_input = input("모델을 저장하시겠습니까? (y/n): ")
if save_model_input.lower() == 'y':
    
    # MODEL_DIR이 정의되지 않은 경우 현재 디렉토리 사용
    try:
        MODEL_DIR = Path(MODEL_DIR)
    except:
        MODEL_DIR = Path('.')
        print(f"⚠️  MODEL_DIR 미정의, 현재 디렉토리 사용: {MODEL_DIR.absolute()}")
    
    MODEL_DIR.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    model_filename = MODEL_DIR / f'best_lgbm_{main_score}_optimized_{timestamp}.pkl'
    params_filename = MODEL_DIR / f'best_params_{main_score}_{timestamp}.json'
    metrics_filename = MODEL_DIR / f'test_metrics_{main_score}_{timestamp}.json'
    results_filename = MODEL_DIR / f'cv_results_{main_score}_{timestamp}.csv'
    
    # 1. 모델 저장
    joblib.dump(best_model, model_filename)
    print(f"✅ 모델 저장: {model_filename}")
    
    # 2. 파라미터 저장
    best_params_serializable = {}
    for key, value in lgbm_random.best_params_.items():
        if isinstance(value, (np.integer, np.floating)):
            best_params_serializable[key] = float(value) if isinstance(value, np.floating) else int(value)
        else:
            best_params_serializable[key] = value
    
    params_info = {
        'best_params': best_params_serializable,
        f'best_{main_score}_score': float(lgbm_random.best_score_),
        'optimization_target': main_score,
        'cv_folds': n_folds,
        'n_iter': n_iter,
        'timestamp': timestamp,
        'class_labels': {
            '0': '일반가 (다수)',
            '1': '가격미달 (소수)'
        },
        'minority_count': int(minority_count),
        'majority_count': int(majority_count),
        'scale_pos_weight': float(scale_pos_weight)
    }
    
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 파라미터 저장: {params_filename}")
    
    # 3. 테스트 성능 저장
    test_metrics_serializable = {
        key: float(value) if isinstance(value, (np.floating, np.integer)) else value
        for key, value in test_metrics.items()
    }
    
    metrics_info = {
        'test_metrics': test_metrics_serializable,
        'confusion_matrix': {
            'tn': int(tn),  # 일반가 → 일반가 (정답)
            'fp': int(fp),  # 일반가 → 가격미달 (오탐)
            'fn': int(fn),  # 가격미달 → 일반가 (미탐, 치명적!)
            'tp': int(tp)   # 가격미달 → 가격미달 (정답)
        },
        'business_interpretation': {
            'total_minority': int(tp + fn),      # 실제 가격미달 개수
            'detected_minority': int(tp),        # 검출된 가격미달
            'missed_minority': int(fn),          # 놓친 가격미달 (치명적)
            'recall_percentage': float(tp/(tp+fn)*100) if (tp+fn) > 0 else 0,
            'false_alarms': int(fp)              # 일반가을 가격미달로 오판
        },
        'timestamp': timestamp
    }
    
    with open(metrics_filename, 'w', encoding='utf-8') as f:
        json.dump(metrics_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 테스트 성능 저장: {metrics_filename}")
    
    # 4. CV 결과 저장
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"✅ CV 결과 저장: {results_filename}")
    
    print("\n📦 저장된 파일 요약:")
    print(f"1. 모델: {model_filename.name}")
    print(f"2. 파라미터: {params_filename.name}")
    print(f"3. 성능: {metrics_filename.name}")
    print(f"4. CV결과: {results_filename.name}")
    
else:
    print("모델 저장 생략")

print("\n" + "="*60)
print("🎉 모든 작업 완료!")
print("="*60)

In [ ]:
models = [
    'best_lgbm_smote_random',
    'best_lgbm_smote_20251106_155837',
    'best_lgbm_recall_optimized_20251106_162548',
    'best_lgbm_recall_optimized_20251106_201626',
    'best_lgbm_pr_auc_optimized_20251106_215224'
]

In [ ]:
import joblib
loaded_model = joblib.load(MODEL_DIR / 'best_lgbm_pr_auc_optimized_20251106_215224.pkl')
predictions = loaded_model.predict(X_test)

In [ ]:
loaded_model

In [ ]:
# 테스트셋 성능 평가
y_pred = loaded_model.predict(X_test)
y_prob = loaded_model.predict_proba(X_test)[:, 1]

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, digits=3))

# Balanced Accuracy
bal_acc = balanced_accuracy_score(y_test, y_pred)
print(f"\nBalanced Accuracy (Test Set): {bal_acc:.3f}")

# ROC-AUC (OvO, macro)
y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
roc_auc_ovo = roc_auc_score(y_test_bin, y_prob, average='macro')
print(f"ROC-AUC (OvO, macro, Test Set): {roc_auc_ovo:.3f}")

In [ ]:
model = loaded_model.named_steps['model']

In [ ]:
import shap

# SHAP 적용
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Plot
shap.summary_plot(shap_values, X_test)


In [ ]:
# mean absolute shap value per feature
shap_importance = np.abs(shap_values).mean(axis=0)
print(shap_importance)
shap_df = pd.DataFrame({
    'feature': X_test.columns,
    'shap_importance': shap_importance
}).sort_values(by='shap_importance', ascending=False)

print(shap_df)